# Unknown Sequence Identification with BioPython
This notebool will use the BLAST Online Service (trough Biopython) to identify (align) some unknown sequence of Nucleotides (Consider a complete sequence) to their database.

#### Real life
On real case scenarios it's often rare that some sequence will completly match to a database of previous sequences (Specially for Virus), so those services actually align and return the top sequences that has matching parts.

#### References
* https://blast.ncbi.nlm.nih.gov/Blast.cgi
* https://biopython.org/DIST/docs/api/Bio.Blast.NCBIWWW-module.html
* https://github.com/chris-rands/biopython-coronavirus/blob/master/biopython-coronavirus-notebook.ipynb
* https://medium.com/analytics-vidhya/coronavirus-covid-19-genome-analysis-using-biopython-8b8cb1f4a041
* https://github.com/peterjc/biopython_workshop
* https://www.youtube.com/watch?v=QIZ8QH6JcC8
* https://www.youtube.com/watch?v=8A-msg23u0w&list=PLQaBWYcv0RlZnZYwyAQUf8BdnlTTuHKPb

In [1]:
import os
import sys

from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)
input_file = "unknown-sequence.fa"

Python version: sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
Biopython version: 1.76


#### Load Sequence
Here we load a sequence and print it's content and size. Considering that we completelly sequence this genome.

This sequence is around 30Kb(nucleotides) which is a high indication of a virus.

##### GC Content
GC-content (or guanine-cytosine content) is the percentage of nitrogenous bases in a DNA or RNA molecule that are either guanine (G) or cytosine (C).

In [2]:
record = SeqIO.read(input_file, "fasta")
print('Sequence size:', len(record.seq))
# Just print the first 200 nucleotides
print('Sequence:', str(record.seq)[0:200])

if len(record.seq) < 150000:
    print('Its a Virus because the complete sequence is small')
    
# Print the GC content
# The GC content is 0.38, so the sequence is somewhat AT-rich, but within a 'normal' range.
print("GC content (%)", GC(record.seq))

Sequence size: 29903
Sequence: ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGT
Its a Virus because the complete sequence is small
GC content (%) 37.97277865097148


#### Compare to other genome sequences
We will use Blast to search for this particular sequence.
Let's use BLAST to align the unknown sequence to other annoated sequences in the NCBI nt database, which contains sequences from many different species from accross the tree of life.
This may take ~10 minutes since we are doing an online search against many sequences (for larger queries, it would sensible to run BLAST locally instead; see Bio.Blast.Applications)

In [3]:
# Use BlastN (nucleotide-nucleotide) to look for matching sequences
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
blast_qresult = SearchIO.read(result_handle, "blast-xml")

#### Its the SARS-CoV-2 (Covid19)

In [4]:
# Get description of the top 5
[hit.description for hit in blast_qresult[:5]]

['Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/CHN/SH01/2020, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate BetaCoV/Wuhan/IPBCAMS-WH-03/2019, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 TKYE6182_2020 RNA, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/CHN/105/2020, complete genome']

#### Lets Select the first hit (The organism on BLAST database with biggest similarity)

In [5]:
first_hit = blast_qresult[0]
print(blast_qresult[0])
# Split by | and get the 4th element
NCBI_id = first_hit.id.split('|')[3]
print('NCBI ID:',NCBI_id)

Query: No
       definition line
  Hit: gi|1798174254|ref|NC_045512.2| (29903)
       Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, c...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0   53927.40   29903        [0:29903]              [0:29903]
NCBI ID: NC_045512.2


## Use GenBank to gather more information
We can get more information about the organism found on GenBank, for example:
* Organism Type
* The given molecule type (DNA or RNA)
* The proteins encoded by genes (CDSs) that this particular organism creates.

In [6]:
Entrez.email = "leonardoaraujo.santos@gmail.com"
# Get a handle for the nucleotide database using the NCBI ID we got for the first hit
handle = Entrez.efetch(db="nucleotide", id= NCBI_id, retmode="text", rettype="gb")
genbank_record = SeqIO.read(handle, "genbank")

In [7]:
# Show all metadata available for this ID on GenBank
genbank_record.annotations.keys()

dict_keys(['molecule_type', 'topology', 'data_file_division', 'date', 'accessions', 'sequence_version', 'keywords', 'source', 'organism', 'taxonomy', 'references', 'comment', 'structured_comment'])

In [8]:
print('Organism type:', genbank_record.annotations["organism"], '\nMolecule type:',genbank_record.annotations["molecule_type"])

Organism type: Severe acute respiratory syndrome coronavirus 2 
Molecule type: ss-RNA


In [9]:
feature_types = {feature.type for feature in genbank_record.features}
print(feature_types)
# Select genes that encode protein
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]
print('This virus has:', len(CDSs), 'protein encoding genes')

{"5'UTR", "3'UTR", 'stem_loop', 'source', 'CDS', 'mat_peptide', 'gene'}
This virus has: 12 protein encoding genes


#### List all genes, their respective proteins

In [10]:
for gene in CDSs:
    protein_seq = Seq(gene.qualifiers["translation"][0])
    protein_size = len(protein_seq)
    has_start_codon = protein_seq.startswith("M")
    print('Gene:', gene.qualifiers["gene"][0], 'produces protein:', gene.qualifiers["product"], 'size:', protein_size)
    print('*'*5,'Codon:', protein_seq[0:100], '\n')

Gene: ORF1ab produces protein: ['ORF1ab polyprotein'] size: 7096
***** Codon: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRS 

Gene: ORF1ab produces protein: ['ORF1a polyprotein'] size: 4405
***** Codon: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRS 

Gene: S produces protein: ['surface glycoprotein'] size: 1273
***** Codon: MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNI 

Gene: ORF3a produces protein: ['ORF3a protein'] size: 275
***** Codon: MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAG 

Gene: E produces protein: ['envelope protein'] size: 75
***** Codon: MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV 

Gene: M produces protein: ['membrane glycoprotein'] size: 222
***** Codon: MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACF